# Install Libs

In [110]:
!pip install --upgrade boto3
!pip install wget
!brew install ffmpeg
import IPython.display
IPython.display.clear_output()

# Import Libs

In [121]:
import boto3
import dill
import sys, os, shutil
import pandas as pd
import wget
import subprocess
import ffmpeg

# Define Paths

In [122]:
path_home_dir = os.path.expanduser(os.path.join('~','Desktop'))
path_report_dir = os.path.join(path_home_dir, 'AWS_Rekognition')

# Download Example Video

We will use the [town center dataset](http://www.robots.ox.ac.uk/ActiveVision/Research/Projects/2009bbenfold_headpose/project.html). This is a >100mb .avi file, however AWS rekognition only accepts mp4 and mov files, so we need to convert the avi to mp4

In [130]:
def convert_video(video_input, video_output):
    cmds = ['ffmpeg', '-i', video_input, video_output]
    subprocess.Popen(cmds)

In [137]:
url = 'http://www.robots.ox.ac.uk/ActiveVision/Research/Projects/2009bbenfold_headpose/Datasets/TownCentreXVID.avi'
video_filename = 'example_video.avi'

path_video = os.path.join(path_report_dir,video_filename)

if os.path.isfile(path_video)==False:
    wget.download(url, path_video)
    
path_video = path_video.replace('.avi','.mp4')
video_filename = video_filename.replace('.avi','.mp4')

if os.path.isfile(path_video)==False:
    convert_video(path_video.replace('.mp4','.avi'),path_video )

In [133]:
os.listdir(path_report_dir)

['example_video.avi', 'imgs', 'example_video.mp4']

# Load Access Keys

In [134]:
path_access_keys_file = '../accessKeys.csv'

access_keys = pd.read_csv(path_access_keys_file)

personal_access_key = access_keys['Access key ID'].iloc[0]
secret_access_key = access_keys["Secret access key"].iloc[0]

# Upload Video to Bucket

## Start s3 Client

In [135]:
region = 'us-west-1'
s3_client = boto3.client(service_name = 's3',
                      region_name = region,
                      aws_access_key_id = personal_access_key,
                      aws_secret_access_key = secret_access_key)

## Define Bucket

In [136]:
username = 'jl'
bucket = username + '-' + os.path.split(path_report_dir)[1].lower().replace('_','-')
bucket

'jl-aws-rekognition'

## Upload Video File

In [143]:
# Create Bucket if not alread present
if bucket not in [bucket['Name'] for bucket in s3_client.list_buckets()['Buckets']]:
    s3_client.create_bucket(Bucket = bucket, 
                         CreateBucketConfiguration =  {'LocationConstraint': region} )

#upload if the video isn't already in the bucket
if 'Contents' in s3_client.list_objects(Bucket = bucket).keys():
    if video_filename not in [obj['Key'] for obj in s3_client.list_objects(Bucket=bucket)['Contents']]:
        s3_client.upload_file(path_video, bucket, video_filename)
else:
    s3_client.upload_file(path_video, bucket, video_filename)

In [144]:
#list objects in bucket
objs = [obj['Key'] for obj in s3_client.list_objects(Bucket=bucket)['Contents']]
objs

['example_video.mp4']

# Label Video

## Start rekog_client

In [145]:
rekog_client = client = boto3.client(service_name = 'rekognition',
                                      region_name = region, 
                                      aws_access_key_id = personal_access_key,
                                      aws_secret_access_key = secret_access_key)

## Start & Get Labels

In [146]:
response = rekog_client.start_label_detection(Video={'S3Object': {'Bucket': bucket, 
                                                       'Name': video_filename}})
JobId = response['JobId']

response = rekog_client.get_label_detection(JobId = JobId)
while response['JobStatus'] == 'IN_PROGRESS':
    response = rekog_client.get_label_detection(JobId = JobId)
response

{'JobStatus': 'SUCCEEDED',
 'VideoMetadata': {'Codec': 'h264',
  'DurationMillis': 299960,
  'Format': 'QuickTime / MOV',
  'FrameRate': 25.0,
  'FrameHeight': 1080,
  'FrameWidth': 1920},
 'NextToken': 'ufPr8U161oBk++5g2LypfWJSQeE2VDEjqab2PfG7SwAzkayQzIOg2v37Gv2KS6+lQ3ODvy7/GCeE',
 'Labels': [{'Timestamp': 0,
   'Label': {'Name': 'Apparel',
    'Confidence': 62.29508972167969,
    'Instances': [],
    'Parents': []}},
  {'Timestamp': 0,
   'Label': {'Name': 'Bench',
    'Confidence': 50.89900588989258,
    'Instances': [{'BoundingBox': {'Width': 0.07412579655647278,
       'Height': 0.15367385745048523,
       'Left': 0.7875776886940002,
       'Top': 0.2232276052236557},
      'Confidence': 67.44117736816406}],
    'Parents': [{'Name': 'Furniture'}]}},
  {'Timestamp': 0,
   'Label': {'Name': 'Bicycle',
    'Confidence': 96.00991821289062,
    'Instances': [{'BoundingBox': {'Width': 0.06618474423885345,
       'Height': 0.15809549391269684,
       'Left': 0.46213942766189575,
       '

## Print Labels in Human-Friendly Format

In [151]:
Labels = response['Labels']

Timestamp = -1
for Label in Labels:
    if Timestamp != Label['Timestamp']:
        Timestamp = Label['Timestamp']
        print('Timestamp:', Timestamp)
    print('\t',Label['Label']['Name'])

Timestamp: 0
	 Apparel
	 Bench
	 Bicycle
	 Bike
	 Building
	 City
	 Clothing
	 Furniture
	 Hardhat
	 Helmet
	 Human
	 Machine
	 Moped
	 Motor
	 Motor Scooter
	 Motorcycle
	 Path
	 Pavement
	 Pedestrian
	 Person
	 Road
	 Sidewalk
	 Street
	 Town
	 Transportation
	 Urban
	 Vehicle
	 Vespa
	 Walkway
	 Wheel
Timestamp: 480
	 Apparel
	 Bicycle
	 Bike
	 Building
	 City
	 Clothing
	 Furniture
	 Helmet
	 Human
	 Machine
	 Moped
	 Motor
	 Motor Scooter
	 Motorcycle
	 Path
	 Pavement
	 Pedestrian
	 Person
	 Road
	 Sidewalk
	 Street
	 Town
	 Transportation
	 Urban
	 Vehicle
	 Vespa
	 Walkway
	 Wheel
Timestamp: 1000
	 Apparel
	 Asphalt
	 Bicycle
	 Bike
	 Building
	 Bus Stop
	 Chair
	 City
	 Clothing
	 Furniture
	 Helmet
	 Human
	 Machine
	 Moped
	 Motor
	 Motor Scooter
	 Motorcycle
	 Path
	 Pavement
	 Pedestrian
	 Person
	 Road
	 Sidewalk
	 Street
	 Tarmac
	 Town
	 Transportation
	 Urban
	 Vehicle
	 Vespa
	 Walkway
	 Wheel
Timestamp: 1480
	 Apparel
	 Asphalt
	 Bench
	 Bicycle
	 Bike
	 Bus Stop
	 C